In [237]:
!git commit -am "WIP"
!git push

[binder 6dc0f85] WIP
 1 file changed, 178 insertions(+), 85 deletions(-)
Counting objects: 4, done.
Delta compression using up to 32 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.73 KiB | 1.73 MiB/s, done.
Total 4 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/yoyu777/jupyter_notebooks.git
   3463826..6dc0f85  binder -> binder


In [262]:
!pip install  lxml pandas
from urllib.request import urlopen
from lxml import html, etree
import pandas as pd
import numpy as np
import json

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [114]:
def extractText(ele):
    return ele.text.strip()

In [346]:
def extractHoldingsFromStock(url):
    print(url)
    page=urlopen(url)
    tree = html.fromstring(page.read().decode('utf-8'))
    holding=tree.xpath('//div[@class="holding"]')[0]
    tbody=holding.xpath('//tbody')[0]
    holding={
        'FullName':tbody.xpath('//div[@data-type="full"]'),
        'Abbreviation':tbody.xpath('//div[@data-type="abbreviation"]'),
        'Percentage':tbody.xpath('//td[@data-type="percent"]')
    }
    df = pd.DataFrame(data=holding)
    df=df.apply(lambda s:s.apply(extractText))
    return df

In [347]:
funds=[]
funds.append({
        'Name':'Janus Henderson Global Technology Fund',
        'URL':'https://www.bloomberg.com/quote/JAGTX:US',
        'Allocation':4
    })
funds.append({
        'Name':'iShares Healthcare Innovation UCITS ETF',
        'URL':'https://www.bloomberg.com/quote/HEAL:SW',
        'Allocation':2
    })

In [348]:
df=pd.DataFrame(funds,columns=['Name','URL','Allocation'])
df

,Name,URL,Allocation
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2


In [349]:
df['Holdings']=df.URL.apply(extractHoldingsFromStock)

https://www.bloomberg.com/quote/JAGTX:US
https://www.bloomberg.com/quote/HEAL:SW


In [350]:
sum=df.Allocation.sum()
weights=list(map(lambda v:round(v/sum,2),df.Allocation))
df['OverallWeight']=weights
df[['Name','OverallWeight']]

,Name,OverallWeight
0,Janus Henderson Global Technology Fund,0.67
1,iShares Healthcare Innovation UCITS ETF,0.33


In [351]:
df

,Name,URL,Allocation,Holdings,OverallWeight
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4,FullName Abbreviation...,0.67
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2,FullName Abbreviation...,0.33


In [355]:
def factorInWeights(row):
#     print(row.Name)
    row.Holdings['OverallPercentage']=row.Holdings.Percentage.apply(lambda v:float(v)*row.OverallWeight)
    return 

df.apply(factorInWeights,axis=1)

0    None
1    None
dtype: object

In [357]:
allHoldings=pd.concat(df.Holdings.values)
allHoldings
allHoldings.sort_values(['OverallPercentage', 'Percentage'], ascending=[0,0])

,FullName,Abbreviation,Percentage,OverallPercentage
0,Microsoft Corp,MSFT:US,5.54,3.7118
1,Alphabet Inc,GOOG:US,5.42,3.6314
2,Tencent Holdings Ltd,700:HK,4.19,2.8073
3,salesforce.com Inc,CRM:US,3.82,2.5594
4,Alibaba Group Holding Ltd,BABA:US,3.24,2.1708
5,Adobe Systems Inc,ADBE:US,2.99,2.0033
6,Amazon.com Inc,AMZN:US,2.95,1.9765
7,Microchip Technology Inc,MCHP:US,2.93,1.9631
8,Taiwan Semiconductor Manufactu,2330:TT,2.69,1.8023
9,Amphenol Corp,APH:US,2.62,1.7554
